<a href="https://colab.research.google.com/github/Hamza-Ali0237/HF-Projects/blob/main/Intermediate/Conversational%20AI%20App/Chatbot_With_Memory_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [17]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")

In [18]:
# t2t_pipeline = pipeline(
#     task = "text2text-generation",
#     model = model,
#     tokenizer = tokenizer
# )

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32102, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32102, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [20]:
# store conversation history
history = []

In [ ]:
def generate_response(user_prompt):
    # The user's input or question
    instruction = user_prompt

    # Combine the conversation history into a single context string
    # Each entry in history is formatted as "User: <user_input> Assistant: <assistant_response>"
    context = " ".join([f"User: {u} Assistant: {a}" for u, a in history])

    # Create the prompt by combining the user's instruction and the conversation context
    # The [SEP] token separates the instruction from the context
    prompt = f"{instruction} [SEP] {context}"

    # Tokenize the prompt into input tensors for the model
    inputs = tokenizer(
        prompt,
        return_tensors='pt',  # Return PyTorch tensors
        max_length=1024,      # Truncate if the prompt exceeds the max length
        truncation=True       # Enable truncation
    )

    # Move the input tensors to the same device as the model (CPU or GPU)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate a response using the model
    outputs = model.generate(
        **inputs,
        max_length=256,       # Limit the length of the generated response
        do_sample=True,       # Enable sampling for more diverse responses
        top_p=0.9,            # Use nucleus sampling with a top-p value of 0.9
        temperature=0.7       # Control the randomness of the response
    )

    # Decode the generated response from token IDs to a string
    response = tokenizer.decode(
        outputs[0],
        skip_special_tokens=True  # Remove special tokens from the output
    )

    # Append the user's input and the assistant's response to the conversation history
    history.append((user_prompt, response))

    # Return the generated response
    return response

In [ ]:
if __name__ == "__main__":
    # Inform the user about the chatbot and how to exit
    print("GODEL Chatbot: Type 'exit' to quit.\n")
    
    # Start an infinite loop to keep the conversation going
    while True:
        # Prompt the user for input
        user_input = input("You: ")
        
        # Check if the user wants to exit the chat
        if user_input.lower() in {"exit", "quit"}:
            break
        
        # Generate a response from the chatbot based on the user's input
        reply = generate_response(user_input)
        
        # Display the chatbot's response
        print(f"Bot: {reply}\n")

GODEL Chatbot: Type 'exit' to quit.

You: heyyy, whats up?
Bot: Hi, just got back from the gym.

You: noice man, getting them gains
Bot: I'm just getting back from the gym.

You: I know. what did you do in the gym?
Bot: I just got back from the gym, and I'm still getting those gains.

You: bro. just shut up
Bot: The 'getting them gains' part is funny because I was a bit of a troll.

You: lol
Bot: I'm not a troll, but I do find it funny that you got back from the gym.

You: exit


# Gradio UI

In [23]:
!pip install gradio

In [ ]:
import gradio as gr

# Global conversation history to store the chat between the user and the assistant
ui_history = []

# Function to handle user input and generate a response
def chatbot(user_prompt):
    # Combine the conversation history into a single context string
    context = " ".join([f"User: {u} Assistant: {a}" for u, a in ui_history])

    # Create the prompt by combining the user's input and the conversation context
    prompt = f"{user_prompt} [SEP] {context}"

    # Tokenize the prompt into input tensors for the model
    inputs = tokenizer(
        prompt,
        return_tensors="pt",  # Return PyTorch tensors
        truncation=True,      # Truncate if the prompt exceeds the max length
        max_length=1024       # Limit the input length to 1024 tokens
    )

    # Move the input tensors to the same device as the model (CPU or GPU)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate a response using the model
    outputs = model.generate(
        **inputs,
        max_length=256,       # Limit the length of the generated response
        do_sample=True,       # Enable sampling for more diverse responses
        top_p=0.9,            # Use nucleus sampling with a top-p value of 0.9
        temperature=0.7       # Control the randomness of the response
    )

    # Decode the generated response from token IDs to a string
    response = tokenizer.decode(
        outputs[0],
        skip_special_tokens=True  # Remove special tokens from the output
    )

    # Append the user's input and the assistant's response to the conversation history
    ui_history.append((user_prompt, response))

    # Format the conversation history into a list of messages for the Gradio chatbot UI
    messages = []
    for u, a in ui_history:
        messages.append({"role": "user", "content": u})       # Add user messages
        messages.append({"role": "assistant", "content": a}) # Add assistant responses

    # Return the formatted messages to update the chatbot UI
    return messages

# Function to clear the conversation history
def clear_history():
    ui_history.clear()  # Clear the global conversation history
    return []           # Return an empty list to reset the chatbot UI

# Gradio Interface
with gr.Blocks() as demo:
    # Add a title to the chatbot interface
    gr.Markdown("# 🤖 GODEL Chatbot with Memory")
    
    # Create a chatbot UI that displays messages in a conversational format
    chatbot_ui = gr.Chatbot(type="messages")
    
    # Create a row for the input textbox and buttons
    with gr.Row():
        txt = gr.Textbox(placeholder="Type a message...")  # Input box for user messages
        submit_btn = gr.Button("Send")                    # Button to send messages
        clear_btn = gr.Button("Clear")                    # Button to clear the chat history

    # Link the "Send" button and the textbox to the chatbot function
    submit_btn.click(fn=chatbot, inputs=txt, outputs=chatbot_ui)
    txt.submit(fn=chatbot, inputs=txt, outputs=chatbot_ui)
    
    # Link the "Clear" button to the clear_history function
    clear_btn.click(fn=clear_history, inputs=None, outputs=chatbot_ui)

# Launch the Gradio app
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57bf5bacb7f5c430e2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The app is working fine, but for better responses I would suggest changing the model to a better one.